# Regression Tree Implementation from Scratch: Proof-of Concept
Decision Trees are very flexible models that are very prone to overfitting the training data without constraints in place. Typically, the algorithm can be constrained on by how "deep" the tree is allowed to be, what the minimum required of number of samples in a node is, in order for it to be split and by cost-complexity pruning, which essentially adds a penalty to the objective function, which increases the more flexible the tree is (i.e. the more nodes it has). For the purpose of this project, I stuck with using the second option: A node can only be split if it has at least X samples in it. If it has fewer than this, it becomes a leaf and can not be split further. The objective function to minimize is the mean squared error. Knowledge of Decision Trees is largely assumed and will not be explained here. For a list of sources I used to learn what is implemented here, see the references at the end.

This implementation of the CART algorithm is probably not a very good one from a computational efficiency point of view, but this is just a proof of concept and I am not computer scientist, software engineer or data scientist.

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

In [4]:
df = pd.read_csv('auto-mpg.csv', error_bad_lines=False, index_col='car name') # skip the lines with missing values that cause errors

b'Skipping line 34: expected 9 fields, saw 10\nSkipping line 128: expected 9 fields, saw 10\nSkipping line 332: expected 9 fields, saw 10\nSkipping line 338: expected 9 fields, saw 10\nSkipping line 356: expected 9 fields, saw 10\nSkipping line 376: expected 9 fields, saw 10\n'


In [5]:
df.head()

,%mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
car name,,,,,,,,
chevrolet chevelle malibu,18.0,8,307.0,130.0,3504.0,12.0,70,1
buick skylark 320,15.0,8,350.0,165.0,3693.0,11.5,70,1
plymouth satellite,18.0,8,318.0,150.0,3436.0,11.0,70,1
amc rebel sst,16.0,8,304.0,150.0,3433.0,12.0,70,1
ford torino,17.0,8,302.0,140.0,3449.0,10.5,70,1


## The RegresionTree Class 
The algorithm sorts on every column sequentially, sets a threshold between every two datapoints for a given column (so first between observation 1 and 2, then between 2 and 3, and so on and so forth). For every threshold/split, an average value for the independent variable is computed, along with its MSE. The threshold that reduces the MSE is chosen, the data is split into two partitions and the same procedure is repeated for both of those partitions, which in turn can produce up to 4 new partitons (provided the two partitions created by the first split have enough samples in them to be split again). The processes continues until all nodes too few samples in them to be split further. 

In [77]:
class RegressionTree:

    def __init__(self, min_sample_split):
        self.min_sample_split = min_sample_split

    def fit(self, X, y):

        self.X = X
        self.y = y

        try:
            self.no_cols = self.X.shape[1]

        except:
            self.X = self.X.reshape((len(self.X), 1))
            self.no_cols = 1

        # Have to merge X y so that we can keep track of y when we make conditional selections on X
        self.data = np.c_[self.X, self.y]
        self.tree = self.grow_tree(data=self.data)

    def predict(self, inputs):
        node = self.tree
        while node.left:
            if inputs[node.feature_index] <= node.cutoff:
                node = node.left
            else:
                node = node.right

        return node.predicted_value

    def get_best_split(self, data):

        self.best_cutoffs = np.array([])
        self.lowest_mses = np.array([])

        for i in range(self.no_cols):

            self.cutoff_array = np.array([])
            self.mse_array = np.array([])

            for j in range(len(data[:, i]) - 1):

                self.sorted_data = data[data[:, i].argsort()]
                self.cutoff = np.mean(self.sorted_data[:, i][j:j + 2])

                if (len(self.sorted_data[:, -1]) >= self.min_sample_split):

                    self.average_y_left = np.mean(self.sorted_data[self.sorted_data[:, i] <= self.cutoff][:, -1])

                    # To avoid RuntimeWarnings
                    if np.isnan(self.sorted_data[self.sorted_data[:, i] > self.cutoff][:, -1]).size == 0:
                        break

                    self.average_y_right = np.mean(self.sorted_data[self.sorted_data[:, i] > self.cutoff][:, -1])
                    self.mse_left = np.sum(
                        ((self.sorted_data[self.sorted_data[:, i] <= self.cutoff][:, -1] - self.average_y_left) ** 2))
                    self.mse_right = np.sum(
                        ((self.sorted_data[self.sorted_data[:, i] > self.cutoff][:, -1] - self.average_y_right) ** 2))
                    self.mse = (self.mse_left + self.mse_right)/len(self.sorted_data)
                    self.mse_array = np.append(self.mse_array, self.mse)
                    self.cutoff_array = np.append(self.cutoff_array, self.cutoff)

                else:
                    continue

            try:
                self.best_cutoffs = np.append(self.best_cutoffs, self.cutoff_array[self.mse_array.argmin()])
                self.lowest_mses = np.append(self.lowest_mses, np.min(self.mse_array))

            except:  # Return None None if we didn't split the data due to min_sample_split constraint not being met
                return None, None

        self.the_best_cutoff = self.best_cutoffs[np.argmin(self.lowest_mses)]
        self.best_var_to_split_on = self.lowest_mses.argmin()

        return self.the_best_cutoff, self.best_var_to_split_on

    def grow_tree(self, data):
        
        class Node:  # class to keep track of the growing tree
            def __init__(self, predicted_value):
                self.predicted_value = predicted_value
                self.feature_index = None
                self.cutoff = None
                self.left = None
                self.right = None
                self.left_data = None
                self.right_data = None

        self.predicted_value = np.mean(data[:, -1])
        node = Node(predicted_value=self.predicted_value)
        self.threshold, self.idx = self.get_best_split(data)
        if self.idx is not None:
            self.right_idxs = data[:, self.idx] > self.threshold
            self.left_idxs = data[:, self.idx] <= self.threshold
            node.left_data = data[self.left_idxs]
            node.right_data = data[self.right_idxs]
            node.feature_index = self.idx
            node.cutoff = self.threshold
            node.left = self.grow_tree(data=node.left_data)
            node.right = self.grow_tree(data=node.right_data)
        return node

In [80]:
X = df[['horsepower', 'weight']].values
y = df['%mpg'].values
data = np.c_[X,y]

Note: I was pretty tired of this project by the time I got this far, so it is not very robust or stable. It cannot, for instance, make predictions on an array of samples. I.e. one can only pass individual samples into the TreeRegression class. So, in order to make multiple predictions at once, we will make a very unpythonic loop 

In [88]:
y_test = [
    [1,1],
    [90,2000],
    [128, 3000],
    [80, 2800],
]

test = np.array(test)

model = RegressionTree(min_sample_split=100)
model.fit(X,y)

for i in test:
    res = model.predict(i)
    print(res)

33.67971014492754
25.459420289855075
14.721428571428572
22.32


As a sanity check, let's check these results against scikit-learn's DecisionTreeRegressor.

In [89]:
true_model = DecisionTreeRegressor(min_samples_split=100)
true_model.fit(X,y)
result = true_model.predict(y_test)

for i in result:
    print(i)

33.67971014492754
25.45942028985507
14.721428571428572
22.319999999999997


# References 
Hastie, T., Tibshirani, R., & Friedman, J. (2009). The elements of statistical learning: data mining, inference, and prediction. Springer Science & Business Media.

James, G., Witten, D., Hastie, T., & Tibshirani, R. (2013). An introduction to statistical learning (Vol. 112, p. 18). New York: springer.

